In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from kerastuner import HyperModel, RandomSearch
from statsmodels.tsa.seasonal import seasonal_decompose

# Load the dataset from both sheets
file_path = 'sam9.xlsx'
sheet1 = pd.read_excel(file_path, sheet_name='Sheet_1')
sheet2 = pd.read_excel(file_path, sheet_name='Sheet_2')

# Concatenate the data from both sheets
df = pd.concat([sheet1, sheet2])

# Preprocess the data
df['datetime'] = pd.to_datetime(df['order_date'].astype(str) + ' ' + df['order_time'].astype(str))
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')

# Aggregate the data by date and pizza name to get the total quantity ordered each day for each pizza type
daily_data = df.groupby([pd.Grouper(key='datetime', freq='D'), 'pizza_name', 'pizza_ingredients']).agg({'quantity': 'sum'}).reset_index()

# Function to create the LSTM model for hyperparameter tuning
class LSTMHyperModel(HyperModel):
    def __init__(self, look_back):
        self.look_back = look_back

    def build(self, hp):
        model = Sequential()
        model.add(LSTM(units=hp.Int('units', min_value=50, max_value=200, step=50),
                       return_sequences=True, input_shape=(self.look_back, 1)))
        model.add(Dropout(rate=hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))
        model.add(LSTM(units=hp.Int('units', min_value=50, max_value=200, step=50),
                       return_sequences=True))
        model.add(Dropout(rate=hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))
        model.add(LSTM(units=hp.Int('units', min_value=50, max_value=200, step=50)))
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')))
        return model

# Function to process each pizza type separately
def process_pizza_type(pizza_name, pizza_ingredients, data, seasonalities=[7, 30, 90, 365]):
    # Filter data for the specific pizza type
    pizza_data = data[(data['pizza_name'] == pizza_name) & (data['pizza_ingredients'] == pizza_ingredients)]
    
    # Filter train and test data
    train_data = pizza_data[(pizza_data['datetime'] >= '2010-01-01') & (pizza_data['datetime'] <= '2019-12-31')]
    test_data = pizza_data[(pizza_data['datetime'] >= '2020-01-01') & (pizza_data['datetime'] <= '2020-12-31')]
    
    # Seasonal decomposition for multiple periods
    decompositions = []
    for period in seasonalities:
        decomposition = seasonal_decompose(train_data['quantity'], model='additive', period=period)
        decompositions.append(decomposition)
    
    # Combine decompositions
    combined_trend = sum(decomp.trend.dropna() for decomp in decompositions) / len(decompositions)
    combined_seasonal = sum(decomp.seasonal.dropna() for decomp in decompositions) / len(decompositions)
    combined_residual = sum(decomp.resid.dropna() for decomp in decompositions) / len(decompositions)

    # Prepare the data for the LSTM model
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_scaled = scaler.fit_transform(train_data[['quantity']])
    test_scaled = scaler.transform(test_data[['quantity']])

    def create_dataset(dataset, look_back=1):
        X, Y = [], []
        for i in range(len(dataset)-look_back):
            a = dataset[i:(i+look_back), 0]
            X.append(a)
            Y.append(dataset[i + look_back, 0])
        return np.array(X), np.array(Y)

    look_back = 30  # Look back period
    trainX, trainY = create_dataset(train_scaled, look_back)
    testX, testY = create_dataset(test_scaled, look_back)

    # Reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
    testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

    # Hyperparameter tuning
    tuner = RandomSearch(
        LSTMHyperModel(look_back),
        objective='val_loss',
        max_trials=10,
        executions_per_trial=1,
        directory='hyperparameter_tuning',
        project_name='lstm_pizza_forecast'
    )

    tuner.search(trainX, trainY, epochs=50, batch_size=32, validation_data=(testX, testY), verbose=2)
    
    # Get the optimal hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    
    # Build the model with the optimal hyperparameters and train it
    model = tuner.hypermodel.build(best_hps)
    model.fit(trainX, trainY, epochs=50, batch_size=32, validation_data=(testX, testY), verbose=2)

    # Make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)

    # Inverse transform predictions and actual values
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])
    
    # Calculate accuracy metrics
    trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
    print(f'Train RMSE for {pizza_name}: {trainScore}')
    testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
    print(f'Test RMSE for {pizza_name}: {testScore}')

    mae = mean_absolute_error(testY[0], testPredict[:,0])
    mse = mean_squared_error(testY[0], testPredict[:,0])
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((testY[0] - testPredict[:,0]) / testY[0])) * 100

    print(f"Mean Absolute Error (MAE) for {pizza_name} ({pizza_ingredients}): {mae}")
    print(f"Mean Squared Error (MSE) for {pizza_name} ({pizza_ingredients}): {mse}")
    print(f"Root Mean Squared Error (RMSE) for {pizza_name} ({pizza_ingredients}): {rmse}")
    print(f"Mean Absolute Percentage Error (MAPE) for {pizza_name} ({pizza_ingredients}): {mape}%")

    # Further analysis of accuracy metrics
    accuracy = 100 - mape
    print(f"Accuracy for {pizza_name} ({pizza_ingredients}): {accuracy}%")

    # Prepare data for the individual pizza sheet
    forecast_df = test_data[look_back:].copy()
    forecast_df['Predicted'] = testPredict[:, 0]
    forecast_df['Actual'] = testY[0]
    
    # Error Analysis Plot
    error = forecast_df['Actual'] - forecast_df['Predicted']
    plt.figure(figsize=(15, 6))
    plt.plot(forecast_df['datetime'], error, label='Prediction Error')
    plt.xlabel('Date')
    plt.ylabel('Prediction Error')
    plt.title(f'Prediction Error Over Time for {pizza_name} ({pizza_ingredients})')
    plt.legend()
    plt.show()

    # Plot the results
    plt.figure(figsize=(15, 6))
    plt.plot(forecast_df['datetime'], forecast_df['Actual'], label='Actual')
    plt.plot(forecast_df['datetime'], forecast_df['Predicted'], label='Forecast')
    plt.xlabel('Date')
    plt.ylabel('Quantity')
    plt.title(f'Actual vs Forecasted Pizza Demand for {pizza_name} ({pizza_ingredients})')
    plt.legend()
    plt.show()

    return {
        'pizza_name': pizza_name,
        'pizza_ingredients': pizza_ingredients,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'MAPE': mape,
        'Accuracy': accuracy,
        'forecast_df': forecast_df
    }

# Process each pizza type
results = []
for index, row in daily_data[['pizza_name', 'pizza_ingredients']].drop_duplicates().iterrows():
    pizza_name = row['pizza_name']
    pizza_ingredients = row['pizza_ingredients']
    print(f'Processing pizza type: {pizza_name}')
    result = process_pizza_type(pizza_name, pizza_ingredients, daily_data)
    results.append(result)

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Save the results to an Excel file with multiple sheets
with pd.ExcelWriter('LSTM_season_res1_sam9_final.xlsx', engine='xlsxwriter') as writer:
    # First sheet with the metrics summary
    summary_df = results_df[['pizza_name', 'pizza_ingredients', 'MAE', 'MSE', 'RMSE', 'MAPE', 'Accuracy']]
    summary_df.to_excel(writer, sheet_name='Summary', index=False)
    
    # Individual sheets for each pizza type
    for result in results:
        sheet_name = f"{result['pizza_name'][:30]}"  # Sheet names should not exceed 31 characters
        result['forecast_df'].to_excel(writer, sheet_name=sheet_name, index=False)

# Overall graph of actual vs forecasted pizza demand for all pizzas
overall_actual = []
overall_forecast = []
pizza_labels = []

for result in results:
    overall_actual.append(result['forecast_df']['Actual'].sum())
    overall_forecast.append(result['forecast_df']['Predicted'].sum())
    pizza_labels.append(result['pizza_name'])

x = np.arange(len(pizza_labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(15, 6))
rects1 = ax.bar(x - width/2, overall_actual, width, label='Actual')
rects2 = ax.bar(x + width/2, overall_forecast, width, label='Forecast')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('Pizza Type')
ax.set_ylabel('Total Quantity')
ax.set_title('Overall Actual vs Forecasted Pizza Demand')
ax.set_xticks(x)
ax.set_xticklabels(pizza_labels, rotation=45, ha='right')
ax.legend()

fig.tight_layout()
plt.show()

# Print the metrics for each pizza type
print(results_df)

# Calculate overall metrics
overall_mae = results_df['MAE'].mean()
overall_mse = results_df['MSE'].mean()
overall_rmse = results_df['RMSE'].mean()
overall_mape = results_df['MAPE'].mean()
overall_accuracy = 100 - overall_mape

print(f"Overall Mean Absolute Error (MAE): {overall_mae}")
print(f"Overall Mean Squared Error (MSE): {overall_mse}")
print(f"Overall Root Mean Squared Error (RMSE): {overall_rmse}")
print(f"Overall Mean Absolute Percentage Error (MAPE): {overall_mape}%")
print(f"Overall Accuracy: {overall_accuracy}%")